In [ ]:
# DF = returnRawDFperBenchmark(Benchmark = "is", fix="Classes", benchmarkClass = ["A", "B", "C", "D"], FixedProcess = 64, Processes = [1,2,4,8,16,32,64,128,256], FixedBenchmarkClass="B")
# DF

In [ ]:
# raw_x = DF.columns.tolist()
# functionName = "MPI_Irecv()"
# raw_y = [DF.at[functionName, x] for x in raw_x]

# # グラフのプロット
# fig = plt.figure(figsize=(12, 9))
# ## 実データをsklearn用に変形
# raw_x = np.array(raw_x).reshape(-1,1)
# raw_y = np.array(raw_y).reshape(-1, 1)
# ## 説明変数と目的変数に分割
# notTrain_x, train_x, target_x = raw_x[0], raw_x[1:-1], raw_x[-1]
# notTrain_y, train_y, target_y = raw_y[0], raw_y[1:-1], raw_y[-1]
# ## 実データをそれぞれプロット
# plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
# plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")
# plt.scatter(notTrain_x, notTrain_y, marker="o", label="最初のデータを除外した時に予測に用いなかった関数コール回数")
# ## モデル式をプロットするために変数”plot_x”を用意する
# plot_x_min = 1.0
# plot_x_max = 256.0
# ### モデル式としてプロットする最低値の設定
# plot_x = np.linspace(plot_x_min, plot_x_max, 500)
# plot_x = np.array(plot_x).reshape(-1, 1)
# ## モデルを [1 - 1/n] として計算
# plot_y = 1 - 1/plot_x
# plt.plot(plot_x, plot_y)


# modelIpMk2 = ModelIp_mk2(train_x=train_x, train_y=train_y, target_x=target_x, target_y=target_y)
# modelIpMk2.calc_lr()
# plot_y = modelIpMk2.predict(plot_x)
# plt.plot(plot_x, plot_y)


In [ ]:
# raw_x

# raw_y

# 多変数予測でのExtra-Pへの入力ファイルを作成する

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
benchmarkName = "cg"
fixed = "Process"
benchmarkClasses = ["A", "B", "C", "D"]
FixedProcess = 64
Processes = [1,2,4,8,16,32,64,128,256]
FixedBenchmarkClass = "B"

DF = collectFunctionNamesPerBenchmark(benchmarkName = "cg", benchmarkClasses = benchmarkClasses, processes = Processes)
# for benchmarkClass in benchmarkClasses:
#     for Process in Processes:
#         pass

DF

functionNames = DF["functionName"]
functionNames = tuple(functionNames)
functionNames

print(DF.query('functionName.str.endswith("VECSET")', engine='python'))

# 既存のモデルが意図したとおりに実装できているかを確認する

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
plot_x = np.linspace(1, 30, 50, dtype=float)

### 対数モデル

In [ ]:
# 対数モデルmk2
# ModelBaseを継承した対数モデルはどこかに不具合がある。
# ModelBase2を継承して、改修した対数モデルがこのモデル。

class ModelLog10_mk2(ModelBase2):

    def calc_lr(self):
        self.transformer_log10 = sp.FunctionTransformer(func=np.log10, inverse_func=inverter_log10_func)
        x_train_log10 = self.transformer_log10.transform(self.train_x)
        self.lr = LinearRegression()
        self.lr.fit(x_train_log10, self.train_y)

    def predict(self, num):
        numConverted = self.transformer_log10.transform(num)
        predicted = self.lr.predict(numConverted)
        return(predicted)
    
    def return_coef_(self):
        return self.lr.coef_
    
    def return_intercept_(self):
        return self.lr.intercept_

    def ModelName(self):
        return("ModelLog10_mk2")

# # 使用例
# modelLog10Mk2 = ModelIp_mk2(train_x=train_x, train_y=train_y, target_x=target_x, target_y=target_y)
# modelLog10Mk2.calc_lr()
# plot_y = modelLog10Mk2.predict(plot_x)

In [ ]:
fig = plt.figure(figsize=(12, 9))
# y = -2 * log_10(x) + 3 でプロットする

plot_y_log10_baseLine = -2 * np.log10(plot_x) + 3
plt.plot(plot_x, plot_y_log10_baseLine, label="基準となる線(-2 * log_10(x) + 3)")

# 予測を実行してプロットする
## 作成したデータから学習用データ・テスト用データに切り分ける

train_x, target_x, train_y, target_y = plot_x[:-25], plot_x[-25:], plot_y_log10_baseLine[:-25], plot_y_log10_baseLine[-25:]

model_log10 = ModelLog10(
    train_x, train_y, test_ratio=0
)
plot_x = np.array(plot_x).reshape(-1, 1)
plot_y_log10 = np.array(plot_y_log10_baseLine).reshape(-1, 1)

model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label=f"対数モデル(coef={model_log10.return_coef_()},intercept={model_log10.return_intercept_()})")

# ModelBase2を継承した ModelLog10_mk2 を用いて予測・プロットする
model_log10mk2 = ModelLog10_mk2(train_x=train_x, train_y=train_y, target_x=target_x, target_y=target_y)
model_log10mk2.calc_lr()
plot_y_log10_mk2 = model_log10mk2.predict(plot_x)
plt.plot(plot_x, plot_y_log10_mk2, label=f"対数モデルmk2(coef={model_log10mk2.return_coef_()},intercept={model_log10mk2.return_intercept_()})")

plt.legend()

### 反比例モデル

In [ ]:
# y = -2 * (1/x) + 3 でプロットする

# 予測を実行してプロットする